In [231]:
import requests
import pandas as pd
import numpy as np
from ast import literal_eval
import pickle

"""
Summary of tables and concepts available in 2015 ACS 5-year estimates. 
Also includes detailed variable descriptions, including data collection methods:

http://old.socialexplorer.com/pub/reportdata/metabrowser.aspx?survey=ACS2015_5yr&header=True
"""

API_KEY = 'c8d52047c115ae3d605397c2e2ab846560365abd'

def get_acs_url(variable_names):
    return (
        'http://api.census.gov/data/2015/acs5'
        '?get='
        '%s'
#       '&for=block+group:*&in=state:06+county:037&key='
        '&for=tract:*&in=state:06+county:037&key='
#        '&for=zip+code+tabulation+area:*&in=state:06+county:037&key='
        '%s'
        % (','.join(variable_names), API_KEY)
    )

def get_acs_data(variable_names):
    """
    Retrieves all values for variable_names for all block groups in Los Angeles County.
    Source: American Community Survey, 2015 5-year estimates
    @return Pandas dataframe
    """
    # Split variables into sublists of <= 50 vars to comply with API limits
    # Then, build and merge dataframes with each set of 50 and return the whole.
    var_chunks = [variables[x:x+50] for x in range(0, len(variables), 50)]
    for i, chunk in enumerate(var_chunks):
        url = get_acs_url(chunk)
        data = requests.get(url).json()
        
        if i == 0:
            df = pd.DataFrame(data[1:], columns=data[0])
        else:
            df = pd.merge(df, pd.DataFrame(data[1:], columns=data[0]))

    # Convert cols to numeric types
    non_numeric = ('NAME', 'state', 'county', 'tract', 'block group',
                   'zip code tabulation area')
    for col in df.columns:
        if col not in non_numeric:
            df[col] = pd.to_numeric(df[col])
    return df
    
def get_variable_metadata(variable_name):
    """
    Retrieves metadata associated with variable_name.
    @return dict
    """
    url = ('http://api.census.gov/data/2015/acs5/variables/%s.json' 
           % variable_name)
    return requests.get(url).json()

def table_variables(table_name, number_list):
    """
    Generates list of variables by concatenating table_name with each number in number_list.
    """
    return tuple(
        '%s_%03dE' % (table_name, num)
        for num in number_list
    )

In [164]:
# Variables for 2015 ACS 5-Year Estimates Table
variables = (
    ()
    + ('B01003_001E',) # total population
    + ('B01002_001E',) # median age
    + table_variables('B01001', range(1,50)) # age brackets by sex
    + table_variables('B15003', (1, 21, 22, 23, 24, 25)) # educational attainment (higher ed degrees only)
    + ('B19301_001E',) # per capita income
    + ('B19013_001E',) # median household income
    + table_variables('B19001', range(1,18)) # household income
    + ('B25001_001E',) # total housing units
    + ('B25077_001E',) # median gross rent
    + ('B25064_001E',) # median home value (owner-occupied)
    + table_variables('B25002', range(1,4)) # housing vacancy status
)

In [165]:
# Record metadata for each variable (concept, label, data type)
variable_info = {
    v: get_variable_metadata(v)
    for v in variables
}

# Quick helper function to find variable tags
def find_vars(search_key):
    return [
        variable_info[v]
        for v in variable_info.keys()
        if search_key.lower() in variable_info[v]['concept'].lower()
    ]

In [166]:
# Get DataFrame
df = get_acs_data(variables)

# Create FIPS geoid column
df['geoid'] = df['state'] + df['county'] + df['tract'] # + df['block group']

In [167]:
# Determine which columns contain null values
from pprint import pprint
for v in df.columns[df.isnull().any()]:
    pprint(variable_info[v])

{'concept': 'B01002.  Median Age by Sex',
 'label': 'Median age --!!Total:',
 'name': 'B01002_001E',
 'predicateType': 'int'}
{'concept': 'B19301. Per Capita Income in the Past 12 Months (in 2015 '
            'Inflation-Adjusted Dollars)',
 'label': 'Per capita income in the past 12 months (in 2015 Inflation-adjusted '
          'dollars)',
 'name': 'B19301_001E',
 'predicateType': 'int'}
{'concept': 'B19013. Median Household Income in the Past 12 Months (in 2015 '
            'Inflation-Adjusted Dollars)',
 'label': 'Median household income in the past 12 months (in 2015 '
          'Inflation-adjusted dollars)',
 'name': 'B19013_001E',
 'predicateType': 'int'}
{'concept': 'B25077.  Median Value (Dollars) for Owner-Occupied Housing Units',
 'label': 'Median value (dollars)',
 'name': 'B25077_001E',
 'predicateType': 'int'}
{'concept': 'B25064.  Median Gross Rent (Dollars)',
 'label': 'Median gross rent',
 'name': 'B25064_001E',
 'predicateType': 'int'}


In [168]:
# Total population within tracts that contain at least 1 null value
df[pd.isnull(df).any(axis=1)]['B01003_001E'].sum()

353556

In [169]:
# Percent of population within tracts that contain at least 1 null value
df[pd.isnull(df).any(axis=1)]['B01003_001E'].sum() / df['B01003_001E'].sum()

0.035220395944049979

In [170]:
# Add neighborhood relationships to dataframe
data = [[t.geoid, t.neighborhood_id]
        for t in Tract.objects.all()]
df2 = pd.DataFrame(data, columns=('geoid', 'neighborhood_id'))
df = pd.merge(df, df2)

# Pickle the data and metadata
census_data = {
    'variable_info': variable_info,
    'dataframe': df
}
pickle.dump(census_data, open('../pickles/census_data.p', 'wb'))

In [171]:
# Test an aggregation: estimate the population of Beverly Hills
print('Tract population sum:', df[df['neighborhood_id'] == 35]['B01003_001E'].sum())
print('LA Times:', Neighborhood.objects.get(pk=35).fixed_data)

Tract population sum: 30892
LA Times: {'population_latimes_2000_census': 33829}


In [365]:
TRACT_DATA = pickle.load(open('../pickles/census_data.p', 'rb'))
def _neighborhood_weighted_avg(total_field, value_fields, neighborhood_id=None, make_percent=False):
    """
    Compute weighted average of statistic for neighborhood, by aggregating from tract-level data.
    If no neighborhood_id is supplied, aggregates all tracts in all neighborhoods.
    Note: returns None if total_field column summation is not > 0
    @param total_field: the applicable totals field (e.g., total population in tract)
    @param value_fields: a list of variable names to be added together to produce the value of interest. (For a single value, use a list of one element.)
    @param neighborhood_id: the neighborhood id to select
    @param make_percent: convert count-based target values to decimal fractions
    """
    df = TRACT_DATA
    if neighborhood_id:
        nhood = df[df['neighborhood_id'] == neighborhood_id] # select all rows with neighborhood_id
    else:
        nhood = df # select all rows
    weights = nhood[total_field] / nhood[total_field].sum()
    if not nhood[total_field].sum() > 0:
        return None

    # Produce a 'values' series by adding all the values in value_fields
    values = nhood[value_fields[0]].copy()
    for field in value_fields[1:]:
        values += nhood[field]

    if (make_percent):
        # Return the weighted sum of all target values, each divided by the applicable total
        return_val = ((values / nhood[total_field]) * weights).sum()
    else:
        # Return the weighted sum of all target values
        return_val = (values * weights).sum()
    if np.isnan(return_val):
        return None
    else:
        return return_val


In [367]:
"""
Compute stats for neighborhoods. To avoid a lot of boilerplate code,
we'll define a dict that encapsulates the settings for each stat
that's automatable. (Some others may require a custom approach.)

Totals fields:

'B25001_001E' # total num. of housing units
'B25002_001E' # total housing units for vacancy status
'B01003_001E' # total population
'B15003_001E' # total pop for educational attainment
'B19001_001E' # total num. of households

Target fields:

'B01002_001E' # median age
'B19301_001E' # per capita income
'B15003_021E' # educat attain: assoc degree (%)
'B15003_022E' # educat attain: bachelor's degree (%)
'B15003_023E' # educat attain: masters degree (%)
'B15003_024E' # educat attain: professional school (%)
'B15003_025E' # educat attain: doctorate degree (%)
'B19013_001E' # median household income

'B25064_001E' # median gross rent
'B25064_001E' # median home value (owner-occupied)
'B25002_003E' # vacant homes (%)

"""

age_vars = [
    ['B01001_003E', 'B01001_004E', 'B01001_005E', 'B01001_006E', 'B01001_027E',
     'B01001_028E', 'B01001_029E', 'B01001_030E'], # 0-17
    ['B01001_007E', 'B01001_008E', 'B01001_009E', 'B01001_010E', 'B01001_011E', 'B01001_012E',
     'B01001_031E', 'B01001_032E', 'B01001_033E', 'B01001_034E', 'B01001_035E', 'B01001_036E'], # 18-34
    ['B01001_013E', 'B01001_014E', 'B01001_015E', 'B01001_037E', 'B01001_038E', 'B01001_039E'], # 35-49
    ['B01001_016E', 'B01001_017E', 'B01001_018E', 'B01001_019E', 'B01001_040E', 'B01001_041E',
     'B01001_042E', 'B01001_043E'], # 50-64
    ['B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E',
     'B01001_044E', 'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E', 'B01001_049E'] # 65+
]

stats_meta = {
    'median_age': {
        'total_field': 'B01003_001E',
        'target_fields': ['B01002_001E',],
        'make_percent': False
    },
    'percent_age_0_17': {
        'total_field': 'B01003_001E',
        'target_fields': age_vars[0],
        'make_percent': True
    },
    'percent_age_18_34': {
        'total_field': 'B01003_001E',
        'target_fields': age_vars[1],
        'make_percent': True
    },
    'percent_age_35_49': {
        'total_field': 'B01003_001E',
        'target_fields': age_vars[2],
        'make_percent': True
    },
    'percent_age_50_64': {
        'total_field': 'B01003_001E',
        'target_fields': age_vars[3],
        'make_percent': True
    },
    'percent_age_65_up': {
        'total_field': 'B01003_001E',
        'target_fields': age_vars[4],
        'make_percent': True
    },    
    'per_capita_income': {
        'total_field': 'B01003_001E',
        'target_fields': ['B19301_001E',],
        'make_percent': False
    },
    'median_household_income': {
        'total_field': 'B19301_001E',
        'target_fields': ['B19013_001E',],
        'make_percent': False
    },
    'median_gross_rent': {
        'total_field': 'B25001_001E',
        'target_fields': ['B25064_001E',],
        'make_percent': False
    },
    'median_home_value_owner_occupied': {
        'total_field': 'B25001_001E',
        'target_fields': ['B25077_001E'],
        'make_percent': False
    },
    'percent_homes_vacant': {
        'total_field': 'B25002_001E',
        'target_fields': ['B25002_003E',],
        'make_percent': True
    },
    'percent_associate_degree': {
        'total_field': 'B15003_001E',
        'target_fields': ['B15003_021E',],
        'make_percent': True
    },
    'percent_bachelors_degree': {
        'total_field': 'B15003_001E',
        'target_fields': ['B15003_022E',],
        'make_percent': True
    },
    'percent_masters_degree': {
        'total_field': 'B15003_001E',
        'target_fields': ['B15003_023E',],
        'make_percent': True
    },
    'percent_professional_degree': {
        'total_field': 'B15003_001E',
        'target_fields': ['B15003_024E',],
        'make_percent': True
    },
    'percent_doctoral_degree': {
        'total_field': 'B15003_001E',
        'target_fields': ['B15003_025E',],
        'make_percent': True
    },
    'percent_bachelors_or_higher': {
        'total_field': 'B15003_001E',
        'target_fields': ['B15003_022E', 'B15003_023E', 'B15003_024E', 'B15003_025E'],
        'make_percent': True
    }
}

def compute_stats(neighborhood):
    return {
        k: neighborhood_weighted_avg(
            total_field=stats_meta[k]['total_field'],
            target_fields=stats_meta[k]['target_fields'],
            neighborhood_id=neighborhood.id,
            make_percent=stats_meta[k]['make_percent']
        )
        for k in stats_meta.keys()
    }

compute_stats(Neighborhood.objects.get(name="Watts"))

{'median_age': 24.325586944479955,
 'median_gross_rent': 914.80923238328364,
 'median_home_value_owner_occupied': 214130.22381535231,
 'median_household_income': 29416.040328130897,
 'per_capita_income': 9167.1580382632401,
 'percent_age_0_17': 0.37915974263922625,
 'percent_age_18_34': 0.27338190804891566,
 'percent_age_35_49': 0.18645873279645489,
 'percent_age_50_64': 0.10908006306191147,
 'percent_age_65_up': 0.051919553453491842,
 'percent_associate_degree': 0.033411514802849936,
 'percent_bachelors_degree': 0.030358012125503386,
 'percent_bachelors_or_higher': 0.03637651015621543,
 'percent_doctoral_degree': 0.0,
 'percent_homes_vacant': 0.071253715684560237,
 'percent_masters_degree': 0.0040713369031287342,
 'percent_professional_degree': 0.0019471611275833076}

In [368]:
# Compute stats for each Neighborhood, and save on the model
for n in Neighborhood.objects.all():
    stats = compute_stats(n)
    for key, value in stats.items():
        n.fixed_data[key] = value
    n.save()
    print(n.name, '*', end='')

Angeles Crest *Hasley Canyon *Arcadia *Beverly Hills *Castaic *Castaic Canyons *Azusa *Cerritos *Chatsworth *Calabasas *Carson *Culver City *Claremont *Compton *Covina *East San Gabriel *Glendale *Florence *East Compton *East Los Angeles *El Monte *Jefferson Park *Lennox *La Cañada Flintridge *Harbor Gateway *Hancock Park *Irwindale *La Verne *La Mirada *Lancaster *Long Beach *Northeast Antelope Valley *Malibu *Pico-Robertson *Palmdale *Monrovia *Montebello *Elysian Valley *Palos Verdes Estates *Pasadena *Pico Rivera *Rancho Palos Verdes *Redondo Beach *Ramona *Ridge Route *Rolling Hills Estates *Rosemead *San Dimas *San Gabriel *San Pedro *Santa Clarita *Santa Fe Springs *Southeast Antelope Valley *Vermont Vista *Stevenson Ranch *Sun Village *Topanga *Torrance *Tujunga Canyons *Unincorporated Catalina Island *Unincorporated Santa Monica Mountains *Unincorporated Santa Susana Mountains *Northwest Palmdale *Whittier *Westlake Village *West Covina *Wilmington *View Park-Windsor Hills *Po

In [369]:
Neighborhood.objects.first().data

{'avg_estimated_bookings_per_listing_per_month': 4.34049479166666,
 'avg_estimated_revenue_per_listing_per_month': 5307.615234375,
 'avg_host_experience_years': 0.543310082435003,
 'avg_listing_price': 451.0,
 'crime_count': 842,
 'crimes_per_capita': 0.1291015026065624,
 'listing_count': 3,
 'median_age': 45.1756062767475,
 'median_gross_rent': 670.6654991243432,
 'median_home_value_owner_occupied': 415123.50262697024,
 'median_household_income': 83798.29791631277,
 'per_capita_income': 34437.263908701854,
 'percent_age_0_17': 0.20256776034236804,
 'percent_age_18_34': 0.18454156399948127,
 'percent_age_35_49': 0.1990662689664116,
 'percent_age_50_64': 0.27791466735831927,
 'percent_age_65_up': 0.1359097393334198,
 'percent_associate_degree': 0.10245824732595235,
 'percent_bachelors_degree': 0.16513417151435544,
 'percent_bachelors_or_higher': 0.2782886094952149,
 'percent_doctoral_degree': 0.013698630136986302,
 'percent_homes_vacant': 0.09737302977232926,
 'percent_masters_degree': 

In [232]:
q=Listing.objects.filter(price__lt=300)

In [262]:
from django.db.models.expressions import ExpressionWrapper
from django.db.models import FloatField
q.values_list('room_type').distinct().order_by('room_type').annotate(
    percent=ExpressionWrapper(Count('room_type') * 1.0 / q.count(), output_field=FloatField())
)

<QuerySet [{'room_type': 'Entire home/apt', 'percent': 0.5368285345938977}, {'room_type': 'Private room', 'percent': 0.40820799312419426}, {'room_type': 'Shared room', 'percent': 0.05496347228190804}]>

In [280]:
z=q.values_list('room_type').distinct().annotate(percent=ExpressionWrapper(Count('room_type') * 1.0 / q.count(), output_field=FloatField()))

In [295]:
dict(q.values_list('bed_type').distinct().annotate(percent=ExpressionWrapper(Count('room_type') * 1.0 / q.count(), output_field=FloatField())))

{'Airbed': 0.00954018048990116,
 'Couch': 0.006574989256553502,
 'Futon': 0.015040825096691019,
 'Pull-out Sofa': 0.009669101847872797,
 'Real Bed': 0.9591749033089815}

In [338]:
list(q.values_list('amenities__name').distinct().order_by('amenities__name').annotate(
                    percent=ExpressionWrapper(Count('amenities__name') * 1.0 / q.count(), output_field=FloatField())))

[('24-Hour Check-in', 0.32303394929093254),
 ('Air Conditioning', 0.6756338633433605),
 ('Breakfast', 0.10421143102707349),
 ('Buzzer/Wireless Intercom', 0.10739149119037388),
 ('Cable TV', 0.37116458960034376),
 ('Carbon Monoxide Detector', 0.5884400515685432),
 ('Cat(s)', 0.06923076923076923),
 ('Dog(s)', 0.1329608938547486),
 ('Doorman', 0.030984099699183497),
 ('Dryer', 0.7371293510958316),
 ('Elevator in Building', 0.20550064460678985),
 ('Essentials', 0.8186076493339063),
 ('Family/Kid Friendly', 0.5128921357971638),
 ('Fire Extinguisher', 0.4322733132788999),
 ('First Aid Kit', 0.3971207563386334),
 ('Free Parking on Premises', 0.6368285345938978),
 ('Gym', 0.1293940696175333),
 ('Hair Dryer', 0.48831113021057154),
 ('Hangers', 0.5601203266007735),
 ('Heating', 0.8462397937258272),
 ('Hot Tub', 0.16587881392350667),
 ('Indoor Fireplace', 0.1954877524709927),
 ('Internet', 0.7303824666953158),
 ('Iron', 0.46832831972496775),
 ('Kitchen', 0.8908895573700043),
 ('Laptop Friendly Wo

In [332]:
Amenity.objects.get(name='Wireless Internet').listing_set.count()

24860

In [341]:
obj={'one': 1, 'two': 2}
json.loads(json.dumps(obj))

{'one': 1, 'two': 2}

In [344]:
filters = {
    'numerical_range': [
        {
            'attribute_name': 'price',
            'min': None,
            'max': 75.00
        },
    ],
}

r=requests.post('http://localhost:8000/api/listings/stats/', json={'filters': filters})
r.json()

{'listings': {'amenities_distribution': {'array_labels': ['24-Hour Check-in',
    'Air Conditioning',
    'Breakfast',
    'Buzzer/Wireless Intercom',
    'Cable TV',
    'Carbon Monoxide Detector',
    'Cat(s)',
    'Dog(s)',
    'Doorman',
    'Dryer',
    'Elevator in Building',
    'Essentials',
    'Family/Kid Friendly',
    'Fire Extinguisher',
    'First Aid Kit',
    'Free Parking on Premises',
    'Gym',
    'Hair Dryer',
    'Hangers',
    'Heating',
    'Hot Tub',
    'Indoor Fireplace',
    'Internet',
    'Iron',
    'Kitchen',
    'Laptop Friendly Workspace',
    'Lock on Bedroom Door',
    'Other pet(s)',
    'Pets Allowed',
    'Pets live on this property',
    'Pool',
    'Safety Card',
    'Shampoo',
    'Smoke Detector',
    'Smoking Allowed',
    'Suitable for Events',
    'TV',
    'Washer',
    'Washer / Dryer',
    'Wheelchair Accessible',
    'Wireless Internet'],
   'comp_avg': [0.32949750671269656,
    0.6829305715381665,
    0.10322209436133487,
    0.1141542

In [ ]:
"""
Prepare extra columns for ML
"""
age_vars = [
    ['B01001_003E', 'B01001_004E', 'B01001_005E', 'B01001_006E', 'B01001_027E',
     'B01001_028E', 'B01001_029E', 'B01001_030E'], # 0-17
    ['B01001_007E', 'B01001_008E', 'B01001_009E', 'B01001_010E', 'B01001_011E', 'B01001_012E',
     'B01001_031E', 'B01001_032E', 'B01001_033E', 'B01001_034E', 'B01001_035E', 'B01001_036E'], # 18-34
    ['B01001_013E', 'B01001_014E', 'B01001_015E', 'B01001_037E', 'B01001_038E', 'B01001_039E'], # 35-49
    ['B01001_016E', 'B01001_017E', 'B01001_018E', 'B01001_019E', 'B01001_040E', 'B01001_041E',
     'B01001_042E', 'B01001_043E'], # 50-64
    ['B01001_020E', 'B01001_021E', 'B01001_022E', 'B01001_023E', 'B01001_024E', 'B01001_025E',
     'B01001_044E', 'B01001_045E', 'B01001_046E', 'B01001_047E', 'B01001_048E', 'B01001_049E'] # 65+
]

stats_meta = {
    'median_age': {
        'total_field': 'B01003_001E',
        'target_fields': ['B01002_001E',],
        'make_percent': False
    },
    'percent_age_0_17': {
        'total_field': 'B01003_001E',
        'target_fields': age_vars[0],
        'make_percent': True
    },
    'percent_age_18_34': {
        'total_field': 'B01003_001E',
        'target_fields': age_vars[1],
        'make_percent': True
    },
    'percent_age_35_49': {
        'total_field': 'B01003_001E',
        'target_fields': age_vars[2],
        'make_percent': True
    },
    'percent_age_50_64': {
        'total_field': 'B01003_001E',
        'target_fields': age_vars[3],
        'make_percent': True
    },
    'percent_age_65_up': {
        'total_field': 'B01003_001E',
        'target_fields': age_vars[4],
        'make_percent': True
    },    
    'per_capita_income': {
        'total_field': 'B01003_001E',
        'target_fields': ['B19301_001E',],
        'make_percent': False
    },
    'median_household_income': {
        'total_field': 'B19301_001E',
        'target_fields': ['B19013_001E',],
        'make_percent': False
    },
    'median_gross_rent': {
        'total_field': 'B25001_001E',
        'target_fields': ['B25064_001E',],
        'make_percent': False
    },
    'median_home_value_owner_occupied': {
        'total_field': 'B25001_001E',
        'target_fields': ['B25077_001E'],
        'make_percent': False
    },
    'percent_homes_vacant': {
        'total_field': 'B25002_001E',
        'target_fields': ['B25002_003E',],
        'make_percent': True
    },
    'percent_associate_degree': {
        'total_field': 'B15003_001E',
        'target_fields': ['B15003_021E',],
        'make_percent': True
    },
    'percent_bachelors_degree': {
        'total_field': 'B15003_001E',
        'target_fields': ['B15003_022E',],
        'make_percent': True
    },
    'percent_masters_degree': {
        'total_field': 'B15003_001E',
        'target_fields': ['B15003_023E',],
        'make_percent': True
    },
    'percent_professional_degree': {
        'total_field': 'B15003_001E',
        'target_fields': ['B15003_024E',],
        'make_percent': True
    },
    'percent_doctoral_degree': {
        'total_field': 'B15003_001E',
        'target_fields': ['B15003_025E',],
        'make_percent': True
    },
    'percent_bachelors_or_higher': {
        'total_field': 'B15003_001E',
        'target_fields': ['B15003_022E', 'B15003_023E', 'B15003_024E', 'B15003_025E'],
        'make_percent': True
    }
}

# Compute percent-based variables
for key in stats_meta.keys():
    if stats_meta[key]['make_percent']:
        total_field = stats_meta[key]['total_field']
        target_fields = stats_meta[key]['target_fields']
        horizontal_sum = tract_df[target_fields[0]].copy()
        for field in target_fields[1:]:
            horizontal_sum += tract_df[field]
        percent = horizontal_sum / tract_df[total_field]
        tract_df[key] = percent